In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
from typing import TypedDict, Annotated
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

llm = init_chat_model("google_genai:gemini-2.5-flash")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state:State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START,"chatbot_node")
builder.add_edge("chatbot_node",END)

graph = builder.compile()

message = {
    "role": "user",
    "content": "Who walked on the moon first time? print name only."
}
response = graph.invoke({"messages": [message]})

print(response["messages"])


[HumanMessage(content='Who walked on the moon first time? print name only.', additional_kwargs={}, response_metadata={}, id='fad1a1f7-f354-4e65-b0d9-9c9b5d23ef45'), AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--c6be2d0b-d6be-4daf-abfb-771f0c984529-0', usage_metadata={'input_tokens': 13, 'output_tokens': 33, 'total_tokens': 46, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 31}})]


In [17]:
state = None
while True:
    input_message = input("your Question: ")
    if input_message.lower() in {"quit", "exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": input_message}],
        }
    else:
        state["messages"].append({"role": "user", "content": input_message})

    state = graph.invoke(state)
    print("Chatbot: ", state["messages"][-1].content)